In [ ]:
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

class YouTubeChatbot:
    def __init__(self):
        # Replace with your actual Groq API key
        self.API_KEY = "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"
        self.URL = "https://api.groq.com/openai/v1/chat/completions"  # Fixed API endpoint
        
    def extract_video_id(self, youtube_url):
        """Extract video ID from YouTube URL."""
        if "v=" in youtube_url:
            return youtube_url.split("v=")[1].split("&")[0]
        elif "youtu.be" in youtube_url:
            return youtube_url.split("youtu.be/")[1].split("?")[0]
        else:
            raise ValueError("Invalid YouTube URL")

    def get_video_transcript(self, video_id):
        """Fetch transcript from YouTube video."""
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            formatter = TextFormatter()
            return formatter.format_transcript(transcript)
        except Exception as e:
            return f"Error fetching transcript: {e}"

    def answer_question(self, transcript, question):
        """Generate answer using Groq API."""
        try:
            headers = {
                "Authorization": f"Bearer {self.API_KEY}",
                "Content-Type": "application/json"
            }
            
            # Using Groq's recommended model
            payload = {
                "model": "mixtral-8x7b-32768",  # Groq's recommended model
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant. Answer questions based on the provided video transcript. If the answer cannot be found in the transcript, say so clearly."
                    },
                    {
                        "role": "user",
                        "content": f"Transcript:\n{transcript}\n\nQuestion: {question}\n\nProvide a concise answer based only on the information in the transcript."
                    }
                ],
                "temperature": 0.1,  # Lower temperature for more focused answers
                "max_tokens": 500
            }

            response = requests.post(self.URL, headers=headers, json=payload)
            response.raise_for_status()
            result = response.json()
            return result["choices"][0]["message"]["content"].strip()
            
        except requests.exceptions.RequestException as e:
            return f"API Error: {str(e)}"
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def run(self):
        """Main chatbot interaction loop."""
        print("Welcome to YouTube Video Chatbot!")
        print("--------------------------------")
        
        while True:
            try:
                youtube_url = input("\nEnter the YouTube video URL (or 'exit' to quit): ").strip()
                
                if youtube_url.lower() == 'exit':
                    print("Thank you for using YouTube Video Chatbot. Goodbye!")
                    break
                    
                video_id = self.extract_video_id(youtube_url)
                print("\nFetching video transcript...")
                transcript = self.get_video_transcript(video_id)
                
                if transcript.startswith("Error"):
                    print(transcript)
                    continue
                
                print("Transcript fetched successfully! You can now ask questions about the video.")
                print("Type 'new video' to analyze a different video or 'exit' to quit.")
                
                while True:
                    question = input("\nYour question: ").strip()
                    
                    if question.lower() == 'exit':
                        print("Thank you for using YouTube Video Chatbot. Goodbye!")
                        return
                    elif question.lower() == 'new video':
                        break
                    elif question:
                        print("\nGenerating answer...")
                        answer = self.answer_question(transcript, question)
                        print(f"\nAnswer: {answer}")
                    else:
                        print("Please enter a valid question.")
                        
            except ValueError as e:
                print(f"Error: {str(e)}")
            except Exception as e:
                print(f"An unexpected error occurred: {str(e)}")

if __name__ == "__main__":
    bot = YouTubeChatbot()
    bot.run()

Welcome to YouTube Video Chatbot!
--------------------------------

Fetching video transcript...
Transcript fetched successfully! You can now ask questions about the video.
Type 'new video' to analyze a different video or 'exit' to quit.

Generating answer...

Answer: The transcript is of a video tour, led by Kendall Jenner, of her home decorated for the holidays. She discusses her love for cozy, organic, and vintage decorations, many of which are hand-me-downs from her mother. She highlights specific rooms, such as the living room with its real wood fireplace and the dining room with Hermes dishes, and shares the stories behind certain decorations, like the Christopher Radko ornaments and the dollhouse. She also talks about her love for hosting and the joy she finds in welcoming friends into her home during the holiday season.
